## Final Project Submission

Please fill out:
* Student name: Ian Tulienge
* Student pace: Full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:



# Data Understanding
Data understanding is the process of getting to know your data before you start analyzing it. This involves gathering information about the data, exploring its structure, identifying its quality, and assessing its relevance to your analysis objectives.

In [ ]:
#importing the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# Reading the bom movie data set
df=pd.read_csv('bom.movie_gross.csv')
df

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('im.db')

# Create a cursor object
cursor = conn.cursor()

# Execute a SQL query to select all the columns from the movie_basics table
query = "SELECT * FROM movie_basics;"
cursor.execute(query)

# Retrieve the results of the query
results = cursor.fetchall()



In [ ]:
#Reading of data from the movie_basics table in SQLite database using Pandas and displays the resulting DataFrame.
movie_basics = pd.read_sql_query("SELECT * FROM movie_basics", conn)
movie_basics

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('im.db')

# Create a cursor object
cursor = conn.cursor()

# Execute a SQL query to select all the columns from the movie_ratings table
query = "SELECT * FROM movie_ratings;"
cursor.execute(query)

# Retrieve the results of the query
results = cursor.fetchall()



In [ ]:
#Reading of data from the movie_ratings table in SQLite database using Pandas and displays the resulting DataFrame.
movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings", conn)
movie_ratings

In [ ]:
#Merging the two tables together
merged_data = pd.merge(movie_basics, movie_ratings, on='movie_id')
merged_data

In [ ]:
#Renaming primary name to title in the movie_basics and movie_ratings merged data
df1=merged_data #defing merged data as df1
df1 = df1.rename(columns={'primary_title': 'title'})
df1

In [ ]:
#merging df1 data to our bom.movie_gross that is define as df
final_data = pd.merge(df, df1, on='title')
final_data #Defining my new data as final_data

In [ ]:
#Cheaking the shape of our data
final_data.shape

In [ ]:
#Cheaking information about our data
final_data.info()

In [ ]:
#provides information such as count, mean, standard deviation, minimum and maximum values, as well as the quartiles of the data.
final_data.describe()

## Perfoming EDA analysis
It is a way of analyzing, visualizing and summarizing data in order to understand its characteristics and identify patterns and trends.
###  Data cleaning
This involves checking the data for any missing or inconsistent values, and correcting or removing them as necessary.
#### Identifying missing values

In [ ]:
#cheaking for missing values where it returns True for missing values
final_data.isnull()

In [ ]:
#Identfying of missing values and getting the sum of each column
final_data.isnull().sum().sort_values(ascending=False) # sort_values(ascending=False) arranges the values in descending order

#### The following do not require  any fills:
     title
     year
     movie_id
     original_title
     start_year
     averagerating
     numvotes
#### The following are missing data:
     studio
     domestic_gross
     foreign_gross
     runtime_minutes
     genres
     

In [ ]:
#Checking percentage of missing values and arrange it in decending order
percent_missing = final_data.isna().sum().sort_values(ascending=False)* 100 / len(df)
percent_missing

foreign_gross contains more than half of the null values this can be shown by a heatmap below

In [ ]:
# Using heatmap to show null values
sns.heatmap(final_data.isna())

Hence I consider removing foreign_gross from my dataset

In [ ]:
#Dropping foreign_gross from the data
new_data = final_data.drop('foreign_gross', axis=1) 
new_data #Renaming my data as new_data

In [ ]:
#Identifying missing values of runtime_minutes
new_data[new_data.runtime_minutes.isnull() ]

In [ ]:
#Identifying missing values of domestic_gross 
new_data[new_data.domestic_gross.isnull()]

#### Filling out missing values by mean()

In [ ]:
#Filling runtime_minutes missing values with the mean of runtime_minutes (107.2395110670631)
mean_runtime = new_data['runtime_minutes'].mean()
new_data['runtime_minutes'].fillna(mean_runtime, inplace=True)
new_data[new_data.runtime_minutes.isnull() ] # Cheaking if runtime_minutes missing values have been filled

In [ ]:
#Filling  missing values with the mean of domestic_gross (30640331.54076539)
mean_runtime = new_data['domestic_gross'].mean()
new_data['domestic_gross'].fillna(mean_runtime, inplace=True)
new_data[new_data.domestic_gross.isnull() ] # Cheaking if domestic_gross missing values have been filled

 I have fill missing values of runtime_minutes and  domestic_gross missing values with there respectively means ie 
 (107.2395110670631 and 30640331.54076539) 

#### Filling missing values by backward filling method


In [ ]:
new_data.fillna(method='bfill', inplace=True) 
# filling missing values by backward filling method in place, 
#which means that the original DataFrame is being updated with the new values rather than creating a new DataFrame
new_data                                       

In [ ]:
#Cheaking of missing value to ensure they have been filled 
new_data.isnull().sum()

No missing values

#### Cheaking for duplicates

In [ ]:
# checks for duplicates and list them
new_data[final_data.duplicated()]

No duplicates values

## Visualizations

#### Lets look at  the top ten genres that have been watched

In [ ]:
# value_counts() method shows the count of different categories in a given column 
new_data['genres'].value_counts().head(10)

In [ ]:
plt.figure(figsize=(12,6)) # create a figure and axis object
# create a countplot
sns.countplot(x='genres',order=new_data['genres'].value_counts().index[0:10],data=new_data)
plt.title('genres of movie by count') # add a title to the plot

 #####  Here a count plot is displayed  with the genres being displayed 

##### Lets look at the top 10 studios in movie production

In [ ]:
# create a figure and axis object
plt.figure(figsize=(12,6))
# create a countplot
sns.countplot(y='studio',order=new_data['studio'].value_counts().index[0:10], data=new_data)
plt.title('Top 10 studios in productuion of movies ') # add a title to the plot

##### Lets check at major ratings given to bom movies

In [ ]:
new_data.averagerating.value_counts()

In [ ]:
plt.figure(figsize=(12,6)) # create a figure and axis object
# create a countplot
sns.countplot(x='averagerating',order=new_data['averagerating'].value_counts().index[0:10],data=new_data)
plt.title('Ratings of movies on bom vs Count') # add a title to the plot

Most movies have an average of 6.8 and 6.4

#### visualization of  averageratings based on numvotes on bom movies

In [ ]:
# Create a scatter plot
new_data.plot(x='averagerating', y='numvotes', kind='scatter')

# Set the x and y axis labels
plt.xlabel('averageRating')
plt.ylabel('numvotes')

# Set the title of the graph
plt.title('Average Rating vs.Number of Votes')
plt.show()

A strong relationship between averagerating and numvotes a positive relationship, where increases in numvotes associated with increases in averagerating.
In the graph, we can conclude that the public often appreciates the movie and generally gives a score between 5/10


#### Lets find out movie with the maximum averagerating

In [ ]:
new_data.averagerating.max()

In [ ]:
# Sort the movies based on the number of ratings in descending order
sorted_movies = new_data.sort_values('averagerating', ascending=False)

# Select the movie with the highest number of ratings (i.e., the first row in the sorted dataframe)
highest_rated_movie = sorted_movies.iloc[0]

# Print out the title of the highest rated movie and its number of ratings
print('The movie with the highest number of ratings is', highest_rated_movie['title'], 'with', highest_rated_movie['averagerating'], 'ratings.')


The Runaways  is  the highest rated movie in bom ratings

#### Lets  find out movie with maximum number of votes

In [ ]:
# Sort the movies based on the number of votes in descending order
sorted_movies = new_data.sort_values('numvotes', ascending=False)

# Select the movie with the highest number of votes (i.e., the first row in the sorted dataframe)
highest_voted_movie = sorted_movies.iloc[0]

# Print out the title of the highest voted movie and its number of votes
print('The movie with the highest number of votes is', highest_voted_movie['title'], 'with', highest_voted_movie['numvotes'], 'votes.')


Inception is the best movie in bom by votes

#### Lets find out movie genre with the highest ratings

In [ ]:
# Group by genre and calculate the mean rating
genre_ratings = new_data.groupby('genres')['averagerating'].mean()

# Get the genres with the highest rating
highest_rated_genres = genre_ratings[genre_ratings == genre_ratings.max()]

# Print the genres with the highest rating
print('Genres with the highest rating:')
for genre in highest_rated_genres.index:
    print(genre)


Adventure genre is the movie with the highest ratings

#### Lets find out  movie genre with the highest number of votes

In [ ]:
# Group by genre and sum the number of votes
genre_votes = new_data.groupby('genres')['numvotes'].sum()

# Get the genre with the highest number of votes
highest_voted_genre = genre_votes.idxmax()

# Print the genre with the highest number of votes
print(f'Genre with the highest number of votes: {highest_voted_genre}')


 Action,Adventure and Sci-Fi are the highest voted genres

#### Lets find out movie genre with the highest domestic gross


In [ ]:
# Group by genre and sum the domestic gross
genre_gross = new_data.groupby('genres')['domestic_gross'].sum()

# Get the genre with the highest domestic gross
highest_gross_genre = genre_gross.idxmax()

# Print the genre with the highest domestic gross
print(f'Genre with the highest domestic gross: {highest_gross_genre}')


Action,Adventure and Sci-Fi have the highest domestic gross

#### Lets find out  the top 10 genres with average runtime  and average no of votes

In [ ]:
# Group by genre and calculate the mean runtime and mean number of votes
genre_stats = new_data.groupby('genres').agg({'runtime_minutes': 'mean', 'numvotes': 'mean'})

# Get the top 10 genres with the highest average number of votes
top_genres = genre_stats.sort_values(by='numvotes', ascending=False).head(10)

# Reset the index to start from 1
top_genres = top_genres.reset_index().rename(columns={'genres': 'Genre', 'runtime_minutes': 'Average Runtime (minutes)', 'numvotes': 'Average Votes'})
top_genres.index = pd.RangeIndex(start=1, stop=len(top_genres)+1)

# Print the top 10 genres in a table with an index starting from 1
print('Top 10 genres with highest average number of votes and average runtime:')
print(top_genres.to_string(index=True, justify='left'))
